In [ ]:
from preprocess import *
tensor,sr = load_mp3_files("../dataset")

for i in tensor:
    print(f"tensor{i}.shape: {i.shape}")
print(f"Sampling rate: {sr}")


In [ ]:
# make them into batched x,y. 
tensor_stack = torch.cat(tensor,dim=-1)
print(f"tensor_stack.shape: {tensor_stack.shape}")

In [ ]:
ck_len = 8000*20 # for first 10 seconds, we predict the next 10 seconds (sampling rate = 8000)

chunks = create_overlapping_chunks_tensor(tensor_stack,chunk_len=ck_len)
print(chunks.shape) #torch.Size([706, 96000])
x,y = chunks[:,ck_len//2:], chunks[:,ck_len//2:]
print(f"x: {x.shape}")
print(f"y: {y.shape}")


In [ ]:

indices = torch.randperm(x.size(0))

shuffled_x,shuffled_y = x[indices],y[indices]

dSet = {
    'x': shuffled_x[:400,:],
    'y': shuffled_y[:400,:],
    'x_test': shuffled_x[400:,:],
    'y_test': shuffled_y[400:,:],
}
from torch.utils.data import TensorDataset,DataLoader
trainDataset,testDataset = TensorDataset(dSet['x'],dSet['y']),TensorDataset(dSet['x_test'],dSet['y_test'])
dLoader,dLoader_test = DataLoader(trainDataset,batch_size=1,shuffle=True),DataLoader(testDataset,batch_size=1,shuffle=False)

In [5]:
from layers.vaeNet import net
model = net(sequence_length=8000*10,num_blocks=10,activation='swish')
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=3e-4,)
# Training settings
num_epochs = 500
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for inputs, labels in dLoader:
        # Zero the parameter gradients
        print(inputs.shape)
        print(labels.shape)
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Accumulate loss
        running_loss += loss.item()
    
    # Average loss for the epoch
    epoch_loss = running_loss / len(dLoader)
    train_losses.append(epoch_loss)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}')

    if epoch % 5 == 0:
        model.eval() 
        running_loss_test = 0.0

        with torch.no_grad(): 
            for inputs, labels in dLoader_test:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_loss_test += loss.item()
        
        # Average validation loss for the epoch
        epoch_test_loss = running_loss_test / len(dLoader_test)
        test_losses.append(epoch_test_loss)

        # Print validation loss for this epoch
        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {epoch_test_loss:.4f}')

# At the end, you can plot the losses if needed

In [ ]:
'''   
    demo for ensuring architecture.
    from layers.vaeNet import net
    import torch
    import torch.nn as nn
    device = torch.device("mps")
    model = net(sequence_length=8000*10,num_blocks=4,activation='swish').to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.AdamW(model.parameters(),lr=3e-4,)

    example_tensor = torch.ones((1,1,8000*10)).to(device)
    out = model(example_tensor)
'''